# Feature Generation pt.2

This module will determine the characteristics for each executive whom we found in pt.1.  

## Features

This module will extract features from the Bloomberg executive biographies using basic if/then logic:
1. Gender
2. Highest degree attained
3. Awards received?
4. Years in career?
5. Top university education?

In [48]:
# to work with data
import pandas as pd

# to work with regex
import re

# to download
import requests

# to work with HTML tags
from bs4 import BeautifulSoup

# to time functions
import datetime

# to use NaN
import numpy as np

# to pause
import time
import random

# to work with local files
import os

import random

In [3]:
try:
    Company_Exec
except NameError:
    Company_Exec = pd.read_csv('Company_CEO_CFO.csv', index_col=0)

In [4]:
Company_Exec[Company_Exec['name'] == 'ADVANCED MICRO DEVICES INC'].sort_values('fy', ascending=False)

,adsh,cik,name,stprba,period,fy,fp,filed,CEO,CFO
1933,0000002488-18-000042,2488,ADVANCED MICRO DEVICES INC,CA,20171231,2017,FY,20180227,Lisa T. Su,Devinder Kumar
7640,0000002488-17-000043,2488,ADVANCED MICRO DEVICES INC,CA,20161231,2016,FY,20170221,Lisa T. Su,Devinder Kumar
13849,0000002488-16-000111,2488,ADVANCED MICRO DEVICES INC,CA,20151231,2015,FY,20160218,Lisa T. Su,Devinder Kumar
22674,0001193125-15-054362,2488,ADVANCED MICRO DEVICES INC,CA,20141231,2014,FY,20150219,Lisa T. Su,Devinder Kumar
29734,0001193125-14-057240,2488,ADVANCED MICRO DEVICES INC,CA,20131231,2013,FY,20140218,Rory P. Read,Devinder Kumar
36966,0001193125-13-069422,2488,ADVANCED MICRO DEVICES INC,CA,20121231,2012,FY,20130221,Rory P. Read,Devinder Kumar
43784,0001193125-12-075837,2488,ADVANCED MICRO DEVICES INC,CA,20111231,2011,FY,20120224,Rory P. Read,Thomas J. Seifert
47351,0001193125-11-040392,2488,ADVANCED MICRO DEVICES INC,CA,20101231,2010,FY,20110218,NaN,Thomas J. Seifert


Pair down the list so that:
1. the cik has at least 5 filings associated with it
2. the cik has at least 5 entries for CEO and CFO

In [5]:
##### get the counts of adsh (filings), CEOs and CFOs.
adsh_ceo_cfo_count_by_cik = Company_Exec.groupby('cik', as_index=True).count()[['adsh','CEO','CFO']]

# filter them
min_entry_count = 5

cik_to_include = adsh_ceo_cfo_count_by_cik[
                            (adsh_ceo_cfo_count_by_cik['adsh'] > min_entry_count) & #check condition 1
                            (adsh_ceo_cfo_count_by_cik['CEO'] > min_entry_count) & #check cond 2
                            (adsh_ceo_cfo_count_by_cik['CFO'] > min_entry_count)]\
                            .index.values #access the index values (CIK)

# include only filings that meet the filter
Company_Exec_filtered = Company_Exec[
    Company_Exec['cik'].isin(cik_to_include)].sort_values(['cik','fy'], ascending=[True, False]).reset_index()
Company_Exec_filtered.head(3)

,index,adsh,cik,name,stprba,period,fy,fp,filed,CEO,CFO
0,577,0001144204-18-051414,2034,ACETO CORP,NY,20180630,2018,FY,20180928,William C. Kennally III,Rebecca Roof
1,6491,0001144204-17-045100,2034,ACETO CORP,NY,20170630,2017,FY,20170825,Salvatore Guccione,Douglas Roth
2,12927,0001571049-16-017785,2034,ACETO CORP,NY,20160630,2016,FY,20160826,Salvatore Guccione,Douglas Roth


Further pair down this list so that:
1. For each company there have been executives (including the current one) that stayed with the company for 3 years




In [6]:
Unique_CIK_filtered = Company_Exec_filtered['cik'].unique().copy()

Company_Exec_filt_Exec_3yr = []

for CIK in Unique_CIK_filtered:
    # to analyze each CIK individually
    df = Company_Exec_filtered[Company_Exec_filtered['cik'] == CIK].copy()
    df['CEO_3yrs_ago'] = df['CEO'].shift(-3)
    df['CFO_3yrs_ago'] = df['CFO'].shift(-3)
    # is the CxO now still the same CxO as 3 yrs ago?
    CEO_been_with_company_3yr = df['CEO'].iloc[0] == df['CEO_3yrs_ago'].iloc[0]
    CEO_been_with_company_3yr = df['CFO'].iloc[0] == df['CFO_3yrs_ago'].iloc[0]
    if CEO_been_with_company_3yr and CEO_been_with_company_3yr:
        Company_Exec_filt_Exec_3yr.append(df)

Company_Exec_filt_Exec_3yr_df = pd.concat(Company_Exec_filt_Exec_3yr)

Company_Exec_filt_Exec_3yr_df.head(3)

,index,adsh,cik,name,stprba,period,fy,fp,filed,CEO,CFO,CEO_3yrs_ago,CFO_3yrs_ago
14,1933,0000002488-18-000042,2488,ADVANCED MICRO DEVICES INC,CA,20171231,2017,FY,20180227,Lisa T. Su,Devinder Kumar,Lisa T. Su,Devinder Kumar
15,7640,0000002488-17-000043,2488,ADVANCED MICRO DEVICES INC,CA,20161231,2016,FY,20170221,Lisa T. Su,Devinder Kumar,Rory P. Read,Devinder Kumar
16,13849,0000002488-16-000111,2488,ADVANCED MICRO DEVICES INC,CA,20151231,2015,FY,20160218,Lisa T. Su,Devinder Kumar,Rory P. Read,Devinder Kumar


In [7]:
# Acquire unique names (CFO and CEO) from executives.
Unique_CEOs = pd.DataFrame(Company_Exec_filt_Exec_3yr_df['CEO'].unique())
Unique_CEOs.columns = ['name']
Unique_CEOs['title'] = 'CEO'

Unique_CFOs = pd.DataFrame(Company_Exec_filt_Exec_3yr_df['CFO'].unique())
Unique_CFOs.columns = ['name']
Unique_CFOs['title'] = 'CFO'

Names_to_research = pd.concat([Unique_CEOs,Unique_CFOs])
Names_to_research = Names_to_research[Names_to_research['name'].notnull()]

Names_to_research.reset_index(drop=True, inplace=True)

Names_to_research.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 2 columns):
name     3657 non-null object
title    3657 non-null object
dtypes: object(2)
memory usage: 57.2+ KB


In [8]:
def find_bio_urls(row):
    # put together the web url for a random search engine
    query = 'bloomberg executive profiles '+row['name']+' '+row['title']
    query = query.replace(' ', '+')
    
    # polite web scraping: pause and use random search engine
    time.sleep(random.randint(1,3)/2)
    engine_y = 'http://search.yahoo.com/search?p='
    engine_b = 'https://www.bing.com/search?q='
    
    def search_with(engine):
        query_url = engine + query
        print('using',query_url)
        r = requests.get(query_url, timeout=3)
        soup = BeautifulSoup(r.text)    
        url = ''
        # Check all returned links
        for a in soup.find_all('a', href=True):
            valid_url = 'bloomberg.com' in a['href'] \
            and 'Executive' in a.text
            if valid_url:
                # select first hit
                if not url:
                    url = a['href']
                    return url
                
    url = search_with(engine_y)
    if not url:
        url = search_with(engine_b)
    if not url:
        return np.nan
    if url:
        print('found ', url)
        return url

if not 'bio_urls_0_10.csv' in os.listdir('./bio_urls/'):     
    name_count = len(Names_to_research)

    a = 0
    b = 10
    while a < 2010:
        df = Names_to_research[a:b].copy()
        print('copied row count', len(df))
        df['bio_urls'] = df.apply(find_bio_urls, axis=1)
        print('processed',a,'to ',b)
        df.to_csv('./bio_urls/bio_urls_'+str(a)+'_'+str(b)+'.csv')
        print('saved',a,'to ',b)
        a = b
        b += 100
else: 
    print('Bio URLS already downloaded')

Bio URLS already downloaded


In [99]:
# Bloomberg blocks requests.get.
# Use Selenium with Firefox to access the site.
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

def scrape_bio(row):
    url = row['bio_urls']
    try:
        # is there a url? if so, scrape politely.
        if not pd.isnull(url):
            driver.get(url)
            sleep_secs = random.randint(6,8)
            time.sleep(sleep_secs)
        else:
            return np.nan

        # is there a show-more button?
        try: 
            read_more_button = driver.find_element_by_xpath('//a[@onclick="show_more(this);"]')
            read_more_button.click()
            sleep_secs2 = random.randint(1,2)
            time.sleep(sleep_secs2)        
        except NoSuchElementException as exception:
            pass

        # is there a biography?
        try: 
            bio_text = driver.find_element_by_xpath('//*[@itemprop="description"]').text
            bio_text = bio_text.replace("'", " ")
            bio_text = bio_text.replace('"', " ")
        except NoSuchElementException as exception:
            bio_text = np.nan

        # is there an age element?
        try:
            age = driver.find_element_by_xpath('//td[@class="largeDetail"][1]').text
        except NoSuchElementException as exception:
            age = np.nan

        # if the age does not make sense, return NaN
        if type(age) is float or not age.isdigit():
            age = np.nan

        return_result = (bio_text, age)
        print(return_result)

        # if the result does not make sense, return 2x nan
        if len(return_result) == 2:
            return return_result
        else:
            return (np.nan, np.nan)
    except TypeError:
        return (np.nan, np.nan)

# Open a Firefox instance and start scraping
driver = webdriver.Firefox()
# Every new instance of Firefox loses the ad blocker. No idea why.
# Activate it manually and press enter when ready.
driver.get('https://addons.mozilla.org/en-US/firefox/addon/ublock-origin/')
input('Activated ad blocker? [enter]')

bio_url_files = random.sample(os.listdir('./bio_urls/'), len(os.listdir('./bio_urls/')))
for datafile in bio_url_files:
    if datafile.replace('bio_urls', 'bios') not in os.listdir('./bios/') and datafile != '.DS_Store':
        df = pd.read_csv('./bio_urls/'+datafile, index_col=0, header=0)
        df['bios'] = np.nan
        df['age'] = np.nan
        print('starting ', datafile)
        try:
            df[['bios', 'age']] = df.apply(scrape_bio, axis=1, result_type='expand')
        # to avoid the program stopping abruptly
        except TypeError:
            print('Encountered TypeError')
            driver.quit()
        except KeyboardInterrupt:
            driver.quit()
        # Save the results
        df.to_csv('./bios/'+str(datafile).replace('bio_urls', 'bios'))
        # Keep track of what's been processed
        print('processed', datafile)
print('all bios downloaded')
driver.quit()

Activated ad blocker? [enter] or "cancel"cancel
starting  bio_urls_3210_3310.csv
processed bio_urls_3210_3310.csv
starting  bio_urls_710_810.csv


MaxRetryError: ("HTTPConnectionPool(host='127.0.0.1', port=57808): Max retries exceeded with url: /session/440dd0e6-6ba5-ee47-8905-1e91d6915caa/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10357ca58>: Failed to establish a new connection: [Errno 61] Connection refused'))", 'occurred at index 710')

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [94]:
def features_from_bio(row):
    Bio = row['bios']
    Name = row['name']
    if not Bio or type(Bio) != str:
        return [np.nan]*5
    try:
        # Is it the correct Bio?
        if not Name.lower() in Bio.lower():
            return [np.nan]*5
        
        # Determine gender based on pronouns in text
        gender = np.nan
        if 'she' in Bio:
            gender = 'Female'
        elif 'he' in Bio:
            gender = 'Male'


        # Determine degree based on mentions in text
        bachelor = ['B.S.', 'Bachelor of Science', ' BS ', "bachelor’s degree"]
        master = ['M.S.', 'Masters of Science', ' MS ', 'Master', 'MBA', 'M.B.A.']
        doctor = ['Dr.', 'Doctor of']

        degree = np.nan
        if any(title.lower() in Bio.lower() for title in bachelor):
            degree = 'Bachelor'
        if any(title.lower() in Bio.lower() for title in master):
            degree = 'Master'    
        if any(title.lower() in Bio.lower() for title in doctor):
            degree = 'Doctorate'


        # Is there any mention of awards?
        awards = False
        accolades = [' award', ' fortune ', ' forbes ']
        for acc in accolades:
            if acc.lower() in Bio.lower():
                awards = True

        # Assume the earliest year mentioned is the start of their career.    
        years = [int(x) for x in re.findall(r'\d{4}', str(Bio))]
        if years:
            career_start = min(years)
        else:
            career_start = np.nan


        # Is there any mention of a top university 
        # https://www.topuniversities.com/university-rankings/world-university-rankings/2019
        Top25Universities = ['Massachusetts Institute of Technology', 'Stanford University', 'Harvard University',
                          'California Institute of Technology', 'University of Oxford', 'University of Cambridge', 
                          'Swiss Federal Institute of Technology', 'Imperial College London', 'University of Chicago',
                          'University College London', 'National University of Singapore', 'Nanyang Technological University',
                          'Princeton University', 'Cornell University', 'Yale University', 'Columbia University',
                          'Tsinghua University', 'University of Edinburgh', 'University of Pennsylvania', 'University of Michigan',
                          'Johns Hopkins University', 'école polytechnique fédérale de lausanne', 'University of Tokyo',
                             'Australian National University', 'University of Hong Kong'
                          ]

        TopUniversity = False
        for uni in Top25Universities:
            if uni.lower() in Bio.lower():
                TopUniversity = True
    except TypeError:
        return [np.nan]*5        
    
    return gender, degree, awards, career_start, TopUniversity

# if we found the wrong Bio for the name, erase the age.
# Other features are not generated in features_from_bio if
# names do not match.
def verify_name(row):
    if row['name'].lower() in row['bios'].lower():
        return row['age']
    else:
        return np.nan

for datafile in random.sample(os.listdir('./bios/'), len(os.listdir('./bios/'))):
    if not 'processed_'+datafile in os.listdir('./bios_processed/') and datafile != '.DS_Store':
        df = pd.read_csv('./bios/'+datafile, header=0, index_col=0)
        df[['gender', 'degree', 'awards', 'career start', 'top university']] = \
        df.apply(features_from_bio, axis=1, result_type='expand')
        df['age'] = df.apply(verify_name, axis=1)
        df.to_csv('./bios_processed/'+'processed_'+datafile)

In [97]:
dfs = []
for datafile in os.listdir('./bios_processed/'):
    if datafile != '.DS_Store':
        df = pd.read_csv('./bios_processed/'+datafile, header=0, index_col=0)
        dfs.append(df)

exec_features = pd.concat(dfs)

exec_features[exec_features['bios'].notnull()].sample(10)

,name,title,bio_urls,bios,age,gender,degree,awards,career start,top university
581,Mark A. Klein,CEO,https://www.bloomberg.com/research/stocks/priv...,Mr. Mark A. Klein serves as the President and ...,63.0,Male,Master,False,1976.0,False
93,Keith S. Walters,CEO,https://www.bloomberg.com/research/stocks/peop...,Mr. Keith S. Walters has been the President at...,68.0,Male,NaN,False,1997.0,False
2430,Brandon S. Pedersen,CFO,https://www.bloomberg.com/research/stocks/peop...,"Mr. Brandon S. Pedersen, CPA, served as Chief ...",51.0,Male,NaN,False,2003.0,False
3621,John D. Kerr,CFO,https://www.bloomberg.com/research/stocks/peop...,Mr. John D. Kerr has been the President and Ch...,51.0,Male,NaN,False,2010.0,False
3623,David M. Duckworth,CFO,https://www.bloomberg.com/research/stocks/peop...,Mr. David M. Duckworth has been the Chief Fina...,38.0,Male,NaN,False,2002.0,False
43,Matthew K. Rose,CEO,https://www.bloomberg.com/research/stocks/priv...,"Mr. Matthew K. Rose, also known as Matt, has b...",58.0,Male,Bachelor,False,1981.0,False
670,Joel H. Reichman,CEO,https://www.bloomberg.com/research/stocks/priv...,"Mr. Joel H. Moser, also known as Joel, Esq. is...",NaN,NaN,NaN,NaN,NaN,NaN
674,Grant C. Bennett,CEO,https://www.bloomberg.com/research/stocks/peop...,"Mr. Grant C. Bennett has been the President, C...",63.0,Male,Master,False,1985.0,True
2237,Randall R. Harwood,CFO,https://www.bloomberg.com/research/stocks/priv...,"Mr. Robert C. Lyons, also known as Bob, has be...",62.0,NaN,NaN,NaN,NaN,NaN
2594,Gregory J. Heinlein,CFO,https://www.bloomberg.com/research/stocks/priv...,"Mr. Gregory J. Heinlein, also known as Greg, h...",54.0,Female,Master,False,1987.0,False


In [96]:
exec_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 857 entries, 0 to 709
Data columns (total 10 columns):
name              857 non-null object
title             857 non-null object
bio_urls          745 non-null object
bios              630 non-null object
age               559 non-null float64
gender            348 non-null object
degree            221 non-null object
awards            353 non-null object
career start      346 non-null float64
top university    353 non-null object
dtypes: float64(2), object(8)
memory usage: 73.6+ KB
